In [1]:
import numpy as np
from tqdm import tqdm
import os

io_path = '/dk/lcj_rag/pyserini-autofusion/experiments/runs'

depth=1000
#  alpha*bf + (1-alpha)*bm25
alpha = 0.5
nq = -1
k=100

bf_trec_path = f'{io_path}/base_trec/run.dpr.wq-train.bm25.trec'
bm25_trec_path = f'{io_path}/base_trec/run.dpr.wq-train.multi.bf.trec'
cache_path = '/dk/lcj_rag/pyserini-autofusion/experiments/cache/'
fusion_tag = 'fusion'


In [3]:

# for alpha in tqdm([0, 0.5, 1.00]):
for alpha in tqdm([0, 1.00]):
    alpha = round(alpha, 2)
    # alpha = f'{alpha:.2f}'
    print(alpha)
    train_out_trec_path = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
    train_out_json_path = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
    cmd0 = f'python -m pyserini.autofusion \
            --runs {bf_trec_path} {bm25_trec_path} \
            --runtag {fusion_tag} \
            --method interpolation  \
            --alpha {alpha} \
            --depth {depth}  \
            --nq {nq}  \
            --k {k} \
            --output {train_out_trec_path}'
    # print(cmd0)
    os.system(cmd0)


  0%|          | 0/2 [00:00<?, ?it/s]

0
num_queries: 87599000


 50%|█████     | 1/2 [1:17:02<1:17:02, 4622.10s/it]

1.0
num_queries: 87599000


100%|██████████| 2/2 [2:32:27<00:00, 4573.63s/it]  


In [2]:
nq = -1
# for alpha in tqdm([0]):
for alpha in tqdm([0, 1.00]):
    # alpha = round(alpha, 2)
    alpha = f'{alpha:.1f}'
    # alphafile = f'/dk/lcj_rag/pyserini-autofusion/experiments/runs/wq_train/alpha-{alpha}.csv'
    train_out_trec_path = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
    train_out_json_path = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
    # cmd0 = f'python -m pyserini.autofusion \
    #         --runs {bf_trec_path} {bm25_trec_path} \
    #         --runtag {fusion_tag}-custom \
    #         --method custom  \
    #         --alphafile {alphafile} \
    #         --depth {depth}  \
    #         --nq {nq}  \
    #         --k {k} \
    #         --output {train_out_trec_path}'
    # # print(cmd0)
    # os.system(cmd0)

    # cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run \
    #     --index wikipedia-dpr-100w \
    #     --topics dpr-wq-train \
    #     --input {train_out_trec_path} \
    #     --output {train_out_json_path}'
    # os.system(cmd1)

    cache_path = '/dk/lcj_rag/pyserini-autofusion/experiments/cache/'
    cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run_on_cache \
    --index wikipedia-dpr-100w \
    --topics dpr-wq-train \
    --gt {cache_path}wq_train_bf_qid_gt_set.pkl {cache_path}wq_train_bm25_qid_gt_set.pkl \
    --input {train_out_trec_path} \
    --output {train_out_json_path}'
    os.system(cmd1)
    

  0%|          | 0/1 [00:00<?, ?it/s]

resultPath: /root/.cache/pyserini/topics-and-qrels/topics.dpr.squad.train.txt
checking path: /dk/rag-dataset/indexes/lucene-index.wikipedia-dpr-100w.20210120.d1b9e6...


Jun 29, 2024 6:32:48 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
100%|██████████| 1/1 [21:36<00:00, 1296.83s/it]


In [3]:
filename = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.all-alpha.nq-{nq}.k-{k}.res.test'
train_out_json_path = f'{io_path}/wq_train/run.dpr.wq-train.{fusion_tag}.alpha-?.nq-{nq}.k-{k}.json'

cmd2 = f'python -m pyserini.eval.evaluate_dpr_retrieval_all_alpha \
    --retrieval {train_out_json_path} \
    --topk 100  \
    --traindata False'
res = os.system(cmd2 + f'1> {filename}')
print(filename)

  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1786it [00:00, 17850.10it/s]
3778it [00:00, 18383.31it/s]
 33%|███▎      | 1/3 [00:01<00:02,  1.15s/it]
0it [00:00, ?it/s]
1763it [00:00, 17628.00it/s]
3778it [00:00, 18026.02it/s]
 67%|██████▋   | 2/3 [00:02<00:01,  1.18s/it]
0it [00:00, ?it/s]
1745it [00:00, 17443.61it/s]
3778it [00:00, 17966.38it/s]
100%|██████████| 3/3 [00:03<00:00,  1.29s/it]


/dk/lcj_rag/pyserini-autofusion/experiments/runs/wq_train/run.dpr.wq-train.fusion.all-alpha.nq--1.k-100.res.test
